In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
%cd drive/MyDrive/DSF_Project_data/

[Errno 2] No such file or directory: 'drive/MyDrive/DSF_Project_data/'
/content/drive/MyDrive/DSF_Project_data


In [84]:
import warnings
warnings.filterwarnings("ignore")

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,mean_absolute_error, mean_squared_error, r2_score,roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [86]:
preProcessed_dfs=[]
def preprocess_df(df):
  pre_snap_data=df[df['frameType']=='BEFORE_SNAP']
  selected_columns = pre_snap_data[['gameId', 'playId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir','nflId']]
  preProcessed_dfs.append(selected_columns)

In [87]:
week_1=pd.read_csv('tracking_week_1.csv')
preprocess_df(week_1)
del week_1
gc.collect()

37678

In [88]:
week_2=pd.read_csv('tracking_week_2.csv')
preprocess_df(week_2)
del week_2
gc.collect()

48290

In [89]:
week_3=pd.read_csv('tracking_week_3.csv')
preprocess_df(week_3)
del week_3
gc.collect()

0

In [90]:
week_4=pd.read_csv('tracking_week_4.csv')
preprocess_df(week_4)
del week_4
gc.collect()

0

In [91]:
week_5=pd.read_csv('tracking_week_5.csv')
preprocess_df(week_5)
del week_5
gc.collect()

0

In [92]:
week_6=pd.read_csv('tracking_week_6.csv')
preprocess_df(week_6)
del week_6
gc.collect()

0

In [93]:
week_7=pd.read_csv('tracking_week_7.csv')
preprocess_df(week_7)
del week_7
gc.collect()

0

In [94]:
week_8=pd.read_csv('tracking_week_8.csv')
preprocess_df(week_8)
del week_8
gc.collect()

0

In [95]:
week_9=pd.read_csv('tracking_week_9.csv')
preprocess_df(week_9)
del week_9
gc.collect()

0

In [96]:
combined_df = pd.concat(preProcessed_dfs, ignore_index=True)

In [97]:
del preProcessed_dfs
gc.collect()

0

In [98]:
plays_data=pd.read_csv('plays.csv')
player_play_data=pd.read_csv('player_play.csv')

In [99]:
df_1=plays_data[['passLength','down','playDescription','yardsToGo','quarter','absoluteYardlineNumber','playClockAtSnap','offenseFormation',
                 'preSnapHomeScore','preSnapVisitorScore','homeTeamWinProbabilityAdded','isDropback','possessionTeam','defensiveTeam',
                 'prePenaltyYardsGained',
                 'preSnapHomeTeamWinProbability','preSnapVisitorTeamWinProbability','yardsGained','passResult','unblockedPressure','timeToThrow','expectedPointsAdded','gameId','playId']]
df_2=player_play_data[['passingYards','hadDropback','rushingYards','hadRushAttempt','receivingYards','fumbles','wasTargettedReceiver','yardageGainedAfterTheCatch',
                       'soloTackle','assistedTackle','tackleForALoss','quarterbackHit','sackYardsAsDefense','hadInterception','hadPassReception','nflId','gameId','playId',
                       'causedPressure','forcedFumbleAsDefense','passDefensed','sackYardsAsOffense','halfSackYardsAsDefense']]

In [100]:
df_1.dropna(inplace=True)

In [101]:
features = [
    'passLength', 'down', 'yardsToGo', 'quarter',
    'absoluteYardlineNumber', 'playClockAtSnap', 'unblockedPressure'
]
df_1['completion'] = df_1['passResult'].apply(lambda x: 1 if x == 'C' else 0)
X = df_1[features]
y = df_1['completion']
X.fillna(X.mean(), inplace=True)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
df_1['expectedCompletion'] = model.predict_proba(X_scaled)[:, 1]
df_1['CPOE'] = df_1['completion'] - df_1['expectedCompletion']

In [102]:
merged_df=pd.merge(df_1,df_2,on=['gameId','playId'])

In [103]:
merged_df['YPA'] = merged_df.apply(lambda row: row['passingYards'] / row['hadDropback'] if row['hadDropback'] > 0 else 0, axis=1)
merged_df['pressureOccurred'] = merged_df['unblockedPressure'].apply(lambda x: 1 if x else 0)
total_plays = merged_df.shape[0]
pressure_plays = merged_df['pressureOccurred'].sum()
pressure_rate = pressure_plays / total_plays

In [104]:
features = [
    'passingYards', 'timeToThrow', 'unblockedPressure', 'hadDropback','CPOE','YPA','pressureOccurred','hadInterception'
]
target = 'expectedPointsAdded'

In [105]:
X = merged_df[features]
y = merged_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = LinearRegression()
model.fit(X_train_scaled, y_train)
feature_weights = pd.DataFrame({
    'Feature': features,
    'Weight': model.coef_
}).sort_values(by='Weight', ascending=False)

In [106]:
merged_df['quarter_back_score'] = merged_df[features].apply(
    lambda row: sum(row[feature] * weight for feature, weight in zip(features, model.coef_)), axis=1
)

In [107]:
merged_df['totalYards'] = merged_df['rushingYards'] + merged_df['receivingYards']
merged_df['EPA'] = merged_df['expectedPointsAdded']
merged_df['yardsAfterContactPerAttempt'] = merged_df.apply(
    lambda row: row['yardageGainedAfterTheCatch'] / row['hadRushAttempt'] if row['hadRushAttempt'] > 0 else 0, axis=1
)
merged_df['touches'] = merged_df['hadRushAttempt'] + merged_df['hadPassReception']
games_played = merged_df.groupby('nflId')['gameId'].nunique()
merged_df = merged_df.merge(games_played, on='nflId', suffixes=('', '_totalGames'))
merged_df['rushAttemptsPerGame'] = merged_df['hadRushAttempt'] / merged_df['gameId_totalGames']
merged_df['receivingYardsPerGame'] = merged_df['receivingYards'] / merged_df['gameId_totalGames']

In [108]:
target='totalYards'
features=['EPA','yardsAfterContactPerAttempt','touches','rushAttemptsPerGame','receivingYardsPerGame']
X = merged_df[features]
y = merged_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = LinearRegression()
model.fit(X_train_scaled, y_train)
feature_weights = pd.DataFrame({
    'Feature': features,
    'Weight': model.coef_
}).sort_values(by='Weight', ascending=False)

In [109]:
merged_df['running_back_score'] = merged_df[features].apply(
    lambda row: sum(row[feature] * weight for feature, weight in zip(features, model.coef_)), axis=1
)

In [110]:
average_stats=combined_df.groupby('nflId').agg({'s':'mean','a':'mean'}).reset_index()
average_stats.rename(columns={'s':'averageSpeed','a':'averageAcceleration'},inplace=True)

In [111]:
merged_df=pd.merge(merged_df,average_stats,on='nflId')

In [112]:
del average_stats
gc.collect()

0

In [113]:
receiver_stats = merged_df.groupby('nflId').agg({
    'hadPassReception': 'sum',
    'wasTargettedReceiver': 'sum'
}).reset_index()
receiver_stats['catchRate'] = receiver_stats['hadPassReception'] / receiver_stats['wasTargettedReceiver']
receiver_stats['catchRate'].fillna(0, inplace=True)

In [114]:
receiver_stats=receiver_stats[['nflId','catchRate']]

In [115]:
merged_df=pd.merge(merged_df,receiver_stats,on='nflId')

In [116]:
merged_df['yardsAfterCatch'] = merged_df['yardageGainedAfterTheCatch']
merged_df['totalTouchdowns'] = merged_df['playDescription'].apply(lambda x: 1 if 'touchdown' in x.lower() else 0)

In [117]:
merged_df['speedAccelerationFactor'] = merged_df['averageSpeed'] * merged_df['averageAcceleration']

In [118]:
target='receivingYards'
features=['yardsAfterCatch','totalTouchdowns','speedAccelerationFactor','EPA','catchRate']
X = merged_df[features]
y = merged_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = LinearRegression()
model.fit(X_train_scaled, y_train)
feature_weights = pd.DataFrame({
    'Feature': features,
    'Weight': model.coef_
}).sort_values(by='Weight', ascending=False)

In [119]:
merged_df['wide_receiver_score']=merged_df[features].apply(
    lambda row: sum(row[feature] * weight for feature, weight in zip(features, model.coef_)), axis=1
)

In [120]:
merged_df['explosivenessScore']=merged_df['averageSpeed']*0.6+merged_df['averageAcceleration']*0.4

In [121]:
target='expectedPointsAdded'
features=['receivingYardsPerGame','yardsAfterCatch','catchRate','touches','explosivenessScore']
X = merged_df[features]
y = merged_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = LinearRegression()
model.fit(X_train_scaled, y_train)
feature_weights = pd.DataFrame({
    'Feature': features,
    'Weight': model.coef_
}).sort_values(by='Weight', ascending=False)
merged_df['tight_end_score']=merged_df[features].apply(
    lambda row: sum(row[feature] * weight for feature, weight in zip(features, model.coef_)), axis=1
)

In [122]:
merged_df['offense_score']=merged_df['quarter_back_score']+merged_df['running_back_score']+merged_df['wide_receiver_score']+merged_df['tight_end_score']

In [123]:
game_data=pd.read_csv('games.csv')

In [124]:
game_df=game_data[['gameId','homeFinalScore','visitorFinalScore']]
merged_df=pd.merge(merged_df,game_df,on='gameId')

In [127]:
data = merged_df
features = [
    'down',
    'yardsToGo',
    'quarter',
    'playClockAtSnap',
    'preSnapHomeScore',
    'preSnapVisitorScore',
]
data['firstDown'] = data['yardsGained'] >= data['yardsToGo']
data = data.dropna(subset=features + ['firstDown'])
X = data[features]
y = data['firstDown'].astype(int)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Model Accuracy: {accuracy:.2f}")
print(f"ROC AUC Score: {roc_auc:.2f}")
data['firstDownProbability'] = model.predict_proba(scaler.transform(X))[:, 1]

Model Accuracy: 0.67
ROC AUC Score: 0.63


In [128]:
data = merged_df
features = [
    'down',
    'yardsToGo',
    'quarter',
    'playClockAtSnap',
    'preSnapHomeScore',
    'preSnapVisitorScore',
    'offense_score'
]
data['firstDown'] = data['yardsGained'] >= data['yardsToGo']
data = data.dropna(subset=features + ['firstDown'])
X = data[features]
y = data['firstDown'].astype(int)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
nmodel = LogisticRegression()
nmodel.fit(X_train, y_train)
y_pred = nmodel.predict(X_test)
y_pred_proba = nmodel.predict_proba(X_test)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Model Accuracy: {accuracy:.2f}")
print(f"ROC AUC Score: {roc_auc:.2f}")
data['firstDownProbability'] = nmodel.predict_proba(scaler.transform(X))[:, 1]

Model Accuracy: 0.82
ROC AUC Score: 0.92


In [181]:
data = merged_df
features = [
    'down',
    'yardsToGo',
    'quarter',
    'passingYards',
    'CPOE',
    'YPA',
    'EPA',
    'receivingYards',
    'yardsAfterCatch',
    'explosivenessScore',
    'catchRate',
    'receivingYardsPerGame',
]
data['firstDown'] = data['yardsGained'] >= data['yardsToGo']
data = data.dropna(subset=features + ['firstDown'])
X = data[features]
y = data['firstDown'].astype(int)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
nmodel = LogisticRegression()
nmodel.fit(X_train, y_train)
y_pred = nmodel.predict(X_test)
y_pred_proba = nmodel.predict_proba(X_test)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Model Accuracy: {accuracy:.2f}")
print(f"ROC AUC Score: {roc_auc:.2f}")
data['firstDownProbability'] = nmodel.predict_proba(scaler.transform(X))[:, 1]

Model Accuracy: 0.95
ROC AUC Score: 0.99
